In [1]:
import numpy as np
import pandas as pd
import pickle
import utils
from tqdm import tqdm, trange

In [2]:
all_images = np.load('raw_data/all_images.npy')
all_parametric = pd.read_csv('raw_data/parameter_subset.csv').values
# descriptions_1 = pickle.load(open('descriptions_1.pkl', 'rb'))
# descriptions_2 = pickle.load(open('descriptions_2.pkl', 'rb'))
# descriptions_3 = pickle.load(open('descriptions_3.pkl', 'rb'))


In [3]:
#resize iamges to 80x128
from PIL import Image
resized_images = []
for i in trange(len(all_images)):
    img = Image.fromarray(all_images[i])
    img = img.resize((128, 80))
    resized_images.append(np.array(img))
resized_images = np.array(resized_images)

100%|██████████| 11000/11000 [00:03<00:00, 3525.51it/s]


In [4]:
all_images = resized_images

In [5]:
import sentence_generator
all_parametric_df = pd.read_csv('raw_data/parameter_subset.csv')
descriptions = []
colors = sentence_generator.load_named_colors("colors.txt")
for idx in trange(11000):
    desc = sentence_generator.generate_description(all_parametric_df.iloc[idx], colors)
    descriptions.append(desc)

100%|██████████| 11000/11000 [00:44<00:00, 246.62it/s]


In [6]:
all_images.shape
#move the channell from the 3rd to 1st dim
all_images = np.moveaxis(all_images, -1, 1)


In [7]:
parametric_masked = utils.mask_array_with_nan(all_parametric, fraction=0.7)

In [8]:
masked_images, masks = utils.apply_random_mask(all_images)

In [9]:
from sklearn.model_selection import train_test_split
train_indices, test_indices = train_test_split(np.arange(len(all_images)), test_size=1000, random_state=42)

param_train = parametric_masked[train_indices]
param_test = parametric_masked[test_indices]

masked_train = masked_images[train_indices]
masked_test = masked_images[test_indices]

mask_train = masks[train_indices]
mask_test = masks[test_indices]

images_train = all_images[train_indices]
images_test = all_images[test_indices]

desc_train = [descriptions[i] for i in train_indices]
desc_test = [descriptions[i] for i in test_indices]



In [10]:
data_dir = "data"

# np.save(f"{data_dir}/param_train.npy", param_train)
# np.save(f"{data_dir}/param_test.npy", param_test)

# for i in range(5):
#     #split masked_train into 10
#     np.save(f"{data_dir}/masked_train_{i}.npy", masked_train[i*2000:(i+1)*2000])
# np.save(f"{data_dir}/masked_test.npy", masked_test)

# for i in range(5):
#     #split images into 10
#     np.save(f"{data_dir}/images_train_{i}.npy", images_train[i*2000:(i+1)*2000])
# np.save(f"{data_dir}/images_test.npy", images_test)

# np.save(f"{data_dir}/mask_train.npy", mask_train)
# np.save(f"{data_dir}/mask_test.npy", mask_test)

pickle.dump(desc_train, open(f"{data_dir}/desc_train.pkl", 'wb'))
pickle.dump(desc_test, open(f"{data_dir}/desc_test.pkl", 'wb'))



In [11]:
import numpy as np
from scipy.spatial import distance

def load_named_colors(file_path):
    colors = {}
    with open(file_path, 'r') as file:
        for line in file:
            name, hex_code = line.strip().split('\t')
            rgb = tuple(int(hex_code[i:i+2], 16) for i in (1, 3, 5))  # Convert hex to RGB
            colors[name] = rgb
    return colors

def nearest_named_color(r, g, b, colors):
    target_rgb = (r, g, b)
    nearest_color = None
    min_distance = float('inf')
    
    for name, rgb in colors.items():
        dist = distance.euclidean(target_rgb, rgb)
        if dist < min_distance:
            min_distance = dist
            nearest_color = name
            
    return nearest_color

# Example usage
colors = load_named_colors("colors.txt")
color_name = nearest_named_color(120, 150, 200, colors)
print("Nearest color name:", color_name)


Nearest color name: greyblue
